In [ ]:
# imports
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))

import panel as pn
import holoviews as hv
from bokeh.io import export_svgs, export_png
from holoviews import opts, dim
from holoviews.operation import histogram
hv.extension('bokeh')
from bokeh.resources import INLINE

import importlib
import processing_parameters
import datetime

import functions_bondjango as bd
import functions_plotting as fp
import functions_loaders as fl
import pandas as pd
import numpy as np
from pprint import pprint
import paths
import random
import scipy.stats as stat
import umap
from scipy.signal import medfilt

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
# set up the figure config
importlib.reload(fp)
importlib.reload(processing_parameters)
# define the target saving path
save_path = os.path.join(paths.figures_path, 'Latent_calcium_interaction')

# define the printing mode
save_mode = True
# define the target document
target_document = 'poster'
# set up the figure theme
fp.set_theme()
# load the label dict
label_dict = processing_parameters.label_dictionary
variable_list = processing_parameters.variable_list

In [ ]:
%%time
# load the data

importlib.reload(fl)
importlib.reload(processing_parameters)
# get the paths
path_list, query_list = fl.query_search_list()

# get the data
full_df = []
frame_list = []
meta_list = []

# for all the paths and queries
for path, query in zip(path_list, query_list):

    data, frames, meta = fl.load_preprocessing(path, query, behavior_flag=True)
    
    # for all the trials in data
    for trial in data:
        # TODO: fix this in the actual function
        if 'badFile' in trial.columns:
            continue
        if 'sync_frames' in trial.columns:
            trial = trial.drop(columns=['sync_frames'])
        if 'latent_0' not in trial.columns:
            continue

#         not_cells = [el for el in trial.columns if 'cell' not in el]
#         no_cells_trial = trial[not_cells]
        # remove the nan rows in the latents
        latents = [el for el in trial.columns if 'latent' in el]
        no_nan_vector = ~np.isnan(np.sum(trial[latents].to_numpy(), axis=1))
        no_nan_trial = trial.iloc[no_nan_vector, :]

        full_df.append(no_nan_trial)
        
    frame_list.append(frames)
    meta_list.append(meta)

print(len(full_df))


In [ ]:
# calculate the relationship between calcium and the latents
